In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import string

In [3]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [5]:
import dask

In [6]:
from dask.distributed import Client

client = Client(n_workers=4, threads_per_worker=2, memory_limit="1GB")
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 3.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61591,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 3.73 GiB
Comm: tcp://127.0.0.1:61602,Total threads: 2
Dashboard: http://127.0.0.1:61604/status,Memory: 0.93 GiB
Nanny: tcp://127.0.0.1:61594,


2024-04-22 22:30:11,269 - distributed.scheduler - WARNING - Worker failed to heartbeat for 343s; attempting restart: <WorkerState 'tcp://127.0.0.1:61602', name: 0, status: running, memory: 1, processing: 1>
2024-04-22 22:30:14,035 - distributed.scheduler - WARNING - Worker failed to heartbeat for 343s; attempting restart: <WorkerState 'tcp://127.0.0.1:61603', name: 1, status: running, memory: 1, processing: 1>
2024-04-22 22:30:14,077 - distributed.scheduler - WARNING - Worker failed to heartbeat for 345s; attempting restart: <WorkerState 'tcp://127.0.0.1:61605', name: 2, status: running, memory: 1, processing: 1>
2024-04-22 22:30:14,127 - distributed.scheduler - WARNING - Worker failed to heartbeat for 343s; attempting restart: <WorkerState 'tcp://127.0.0.1:61609', name: 3, status: running, memory: 1, processing: 1>
2024-04-22 22:30:33,277 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2024-04-22 22:30:54,765 - distributed.nanny - WARNING - Worker

In [26]:
import dask.dataframe as dd

df = pd.read_csv('movie.csv')



In [27]:
df = df.drop_duplicates()

In [31]:
df.dropna()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1
...,...,...
39995,"""Western Union"" is something of a forgotten cl...",1
39996,This movie is an incredible piece of work. It ...,1
39997,My wife and I watched this movie because we pl...,0
39998,"When I first watched Flatliners, I was amazed....",1


In [41]:
ddf = dd.from_pandas(df, npartitions=4)

In [8]:
def remove_br_tags(text):
    cleaned_text = text.replace('<br />', '')
    return cleaned_text

In [24]:
def nopunct(mess):
    nopunc = [x for x in mess if x not in string.punctuation]
    nopunc = ''.join(nopunc)
    return nopunc

In [25]:
def remove_stopword(mess):
    stpwrd = [char for char in mess if char not in stopwords.words('english')]
    stpwrd = ''.join(stpwrd)
    return stpwrd

In [42]:
ddf['text'] = ddf['text'].map_partitions(remove_br_tags)

In [43]:
ddf['text_processed'] = ddf['text'].map_partitions(nopunct, meta=('text_processed', str))

In [44]:
ddf['text_nltk_stopwords'] = ddf['text_processed'].map_partitions(remove_stopword, meta=('text_nltk_stopwords', str))

In [47]:
result = ddf.compute()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/distributed/client.py:3161: UserWarning: Sending large graph of size 50.45 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
Python(39764) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(39766) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(39768) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(39769) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
2024-04-22 22:32:58,001 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3577, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/cs/mf3dd9nn20l_gdnk1rvs9wqw0000gp/T/ipykernel_37275/1657180076.py", line 1, in <module>
    result = ddf.compute()
             ^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dask_expr/_collection.py", line 475, in compute
    return DaskMethodsMixin.compute(out, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dask/base.py", line 375, in compute
    (result,) = compute(self, traverse=False, **kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dask/base.py", line 661, in compute
    

In [ ]:
result.head()